In [1]:
# pip install xlrd

import sys 
import os
import pandas as pd
import re

In [2]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [3]:
from config.config import config

In [4]:
df = pd.read_excel(config['services_excel_file'])
df.shape

(10795, 7)

In [5]:
# удаление и переименование

df = df.drop(["Ссылка.Автор.SAMAccount name", "Контрагент"], axis=1)
df.columns = ['link', 'bustype', 'service', 'comment', 'code']
df.shape

(10795, 5)

In [6]:
# преобразование в строку, замена переноса строк на пробел, удаление повторяющихся пробелов 

for col in df.columns:
    df[col] = df[col].apply(lambda x: str(x).replace('\n', ' ').strip())
    df[col] = df[col].apply(lambda x: re.sub(r' {2,}', ' ', x).strip())

In [7]:
# замена "1 в 1"

def replace1in1(row):
    if row['comment'] == '1 в 1':
        return row['service']
    else:
        return row['comment']

df['comment'] = df.apply(replace1in1, axis=1)

In [8]:
df.describe()

,link,bustype,service,comment,code
count,10795,10795,10795,10795,10795
unique,4002,6,637,800,662
top,nan,Логистика,Организация автотранспортной перевозки,Счета авто. Грузятся по реестрам.,190
freq,424,8468,1776,1864,1642


In [9]:
df.describe()

,link,bustype,service,comment,code
count,10795,10795,10795,10795,10795
unique,4002,6,637,800,662
top,nan,Логистика,Организация автотранспортной перевозки,Счета авто. Грузятся по реестрам.,190
freq,424,8468,1776,1864,1642


In [10]:
df.shape

(10795, 5)

### Экспорт списка наиболе часто встречающихся "Услуг" 

In [11]:
service_counts = df['service'].value_counts()

filtered_services = service_counts[service_counts > 25]
filtered_services

service
Организация автотранспортной перевозки                                                                                              1776
Информационные услуги (импортный груз)                                                                                               313
Оказание сопутствующих транспортно-экспедиционных услуг при организации международной перевозки                                      215
Проведение экспертной оценки в целях фитосанитарной экспертизы и установления фитосанитарного состояния подкарантинной продукции     199
Экспертиза, ед                                                                                                                       182
                                                                                                                                    ... 
Организация хранения контейнеров на контейнерном терминале                                                                            27
термоиндикатор                   

In [12]:
import json 

file = os.path.join(config['BASE_DIR'], 'config', 'freq_services.json')
with open(file, 'w', encoding='utf-8') as f:
    json.dump(filtered_services.index.to_list(), f, ensure_ascii=False, indent=4)

### Список частых услуг (для добавления вручную)

In [13]:
# TODO

### Удаление транспортных позиций

In [14]:
def remove_special_characters(text):
    response = re.sub(r'[^\w\s]', ' ', text.lower())
    return re.sub(r'(\s{2,}|\n)', ' ', response)

excluded = ['счета авто грузятся по реестрам', 'услуги грузятся автоматически', 'Счета авто. ']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

['счета авто грузятся по реестрам', 'услуги грузятся автоматически', 'счета авто ']


In [15]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

(7376, 5)

In [16]:
df.describe()

,link,bustype,service,comment,code
count,7376,7376,7376,7376,7376
unique,2496,6,617,798,639
top,nan,Логистика,Проведение экспертной оценки в целях фитосанит...,услуга русс.яз для акта,ТК-003254
freq,424,5049,199,325,199


### Удаление позиций "услуга русс. яз для акта"

In [17]:
excluded = ['услуга русс.яз для акта']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

['услуга русс яз для акта']


In [18]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

(7043, 5)

### Количество различных комментариев в одной услуге

In [ ]:
uniq_services = df.groupby(['service']).agg(num_comments=('comment', 'count'), uniq_comments=('comment', 'nunique')).reset_index()
uniq_services.tail()

In [ ]:
uniq_services.query("uniq_comments > 1")

### Количество различных услуг в одном комментарии

In [ ]:
uniq_comments = df.groupby(['comment']).agg(num_services=('service', 'count'), uniq_services=('service', 'nunique')).reset_index()
uniq_comments.tail()

In [ ]:
uniq_comments.query("uniq_services > 1")

### Услуги с двумя и более кодами

In [ ]:
service_code_counts = df.groupby('service')['code'].nunique()

services_with_multiple_codes = service_code_counts[service_code_counts > 1]
services_with_multiple_codes

In [ ]:
grouped = df[df['service'].isin(services_with_multiple_codes.index)].groupby(['service', 'code']).size().reset_index(name='count')
display(grouped)

idx = grouped.groupby('service')['count'].idxmin()
min_count_code = grouped.loc[idx]
display(min_count_code)

In [ ]:
df_ = df[~df['code'].isin(min_count_code.code)]

In [ ]:
df_.describe()

### Группировка

In [19]:
# old

# grouped_comment = df.groupby(['comment']).agg(
#     service_list=('service', lambda x: list(dict.fromkeys(x))),
#     codelist = ('code', lambda x: list(dict.fromkeys(x)))
# ).reset_index()
# grouped_comment['len_service'] = grouped_comment['service_list'].apply(len)
# grouped_comment['id'] = list(range(1, grouped_comment.shape[0]+1))

In [20]:
df[['comment', 'service', 'code']].head()

,comment,service,code
38,Оформление результатов,Оформление результатов,ТК-009509
39,Дегазация подкарантинной продукции в морском к...,Дегазация подкарантинной продукции в морском к...,ТК-009507
40,1 Изучение представленных документов 2 Оформле...,Лабораторные исследования товара,ТК-008940
41,1 Анализ и первичная экспертиза подкарантинной...,Лабораторные исследования товара,ТК-008940
42,Комплекс услуг: электронное уведомление о приб...,Комплекс услуг: электронное уведомление о приб...,ТК-008270


In [21]:
def custom_agg(group):
    service = list(group['service'])
    code = list(group['code'])
    service_code_zip = list(dict.fromkeys(zip(service, code)))
    service_list, code_list = zip(*service_code_zip)

    return pd.Series({'service_list': list(service_list), 'code_list': list(code_list)})

grouped_comment = df.groupby(['comment']).apply(custom_agg).reset_index()
grouped_comment['len_service'] = grouped_comment['service_list'].apply(len)
grouped_comment['id'] = list(range(grouped_comment.shape[0]))

grouped_comment.head()

,comment,service_list,code_list,len_service,id
0,1 Анализ и первичная экспертиза подкарантинной...,[Лабораторные исследования товара],[ТК-008940],1,0
1,1 Доставка проб в лабораторию. Исследования,[Доставка проб в лабораторию],[ТК-008695],1,1
2,1 Изучение представленных документов 2 Оформле...,[Лабораторные исследования товара],[ТК-008940],1,2
3,15307.35,"[Сборы, связанные с морской перевозкой]",[Т00000671],1,3
4,178.82,[Оформление документации],[Т0778],1,4


In [22]:
error = "Длина service_list всегда должна быть равна длине code_list"

assert grouped_comment[grouped_comment['service_list'].apply(len) != grouped_comment['code_list'].apply(len)].size == 0, error

In [23]:
z = grouped_comment[grouped_comment["comment"] == "Транспортно-экспедиторские услуги ТрансКонтейнер"]
display(z)
display(z['service_list'].to_list())
display(z['code_list'].to_list())
display(len(z['service_list'].to_list()) == len(z['code_list'].to_list()))

,comment,service_list,code_list,len_service,id
621,Транспортно-экспедиторские услуги ТрансКонтейнер,"[Транспортно-экспедиторские услуги (шт), Погру...","[ТК-003562, ТК-005882, ТК-004317, ТК-009584, Т...",6,621


[['Транспортно-экспедиторские услуги (шт)',
  'Погрузо-разгрузочные работы',
  'Досмотр',
  'Перетарка ручная',
  'Хранение (сут)',
  'Замена пломбы']]

[['ТК-003562', 'ТК-005882', 'ТК-004317', 'ТК-009584', 'Т00000332', 'Т1510']]

True

In [24]:
grouped_comment.shape

(796, 5)

In [25]:
# Функция для создания строки формата 'service_list#code_list#'

def create_service_code_string(row):
    return [f"{s}#{c}#" for s, c in zip(row['service_list'], row['code_list'])]

# Применение функции ко всем строкам с помощью apply
grouped_comment['service_code'] = grouped_comment.apply(create_service_code_string, axis=1)
grouped_comment.tail()

,comment,service_list,code_list,len_service,id,service_code
791,хранение 20фут. контейнера импорт с 9-х суток,[Хранение],[ТК-000473],1,791,[Хранение#ТК-000473#]
792,хранение 40фут. контейнера импорт,"[Хранение контейнера, Хранение]","[Т0464, 000000307]",2,792,"[Хранение контейнера#Т0464#, Хранение#000000307#]"
793,хранение 40фут. контейнера импорт контейнера и...,[Хранение],[307],1,793,[Хранение#307#]
794,хранение 40фут. контейнера импорт с 22х суток,[Хранение],[000000307],1,794,[Хранение#000000307#]
795,хранение контейнера импорт,[Хранение контейнера],[Т0464],1,795,[Хранение контейнера#Т0464#]


In [26]:
result_df = grouped_comment[['id', 'comment', 'service_code']]
result_df.tail()

,id,comment,service_code
791,791,хранение 20фут. контейнера импорт с 9-х суток,[Хранение#ТК-000473#]
792,792,хранение 40фут. контейнера импорт,"[Хранение контейнера#Т0464#, Хранение#000000307#]"
793,793,хранение 40фут. контейнера импорт контейнера и...,[Хранение#307#]
794,794,хранение 40фут. контейнера импорт с 22х суток,[Хранение#000000307#]
795,795,хранение контейнера импорт,[Хранение контейнера#Т0464#]


In [27]:
result_df.to_json(config['unique_comments_file'], orient='records', force_ascii=False, indent=4)